In [ ]:
!pip install transformers datasets evaluate pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
import json
from transformers import pipeline
import evaluate

# Load Json files

In [ ]:
with open("train-v1.1.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open("dev-v1.1.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

In [ ]:
print("Train examples:", len(train_data["data"]))
print("Validation examples:", len(dev_data["data"]))

Train examples: 442
Validation examples: 48


# pre-trained QA pipeline

In [ ]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


# Take first question from dev set

In [ ]:
sample_context = dev_data["data"][0]["paragraphs"][0]["context"]
sample_question = dev_data["data"][0]["paragraphs"][0]["qas"][0]["question"]
sample_answer = dev_data["data"][0]["paragraphs"][0]["qas"][0]["answers"][0]["text"]

In [ ]:
print("\nQuestion:", sample_question)
print("Predicted Answer:", qa_pipeline(question=sample_question, context=sample_context)["answer"])
print("True Answer:", sample_answer)


Question: Which NFL team represented the AFC at Super Bowl 50?
Predicted Answer: Denver Broncos
True Answer: Denver Broncos


# Evaluation

In [20]:
metric = evaluate.load("squad")

preds, refs = [], []

# loop over first 200 QAs in dev set for quick evaluation
for article in dev_data["data"][:10]:
    for para in article["paragraphs"]:
        context = para["context"]
        for qa in para["qas"][:2]:
            question = qa["question"]
            qid = qa["id"]
            true_answer = qa["answers"][0]["text"]

            pred = qa_pipeline(question=question, context=context)

            # Prediction format required by squad metric
            preds.append({"id": qid, "prediction_text": pred["answer"]})
            # Reference format
            refs.append({
                "id": qid,
                "answers": {
                    "text": [true_answer],
                    "answer_start": [0]  # dummy index, not used here
                }
            })

results = metric.compute(predictions=preds, references=refs)

print("Exact Match:", results["exact_match"])
print("F1 Score:", results["f1"])


Exact Match: 69.11344137273593
F1 Score: 80.90495540990912
